# 演習の準備
## 必要なライブラリのインポート

工学部システム創成学科Cコース3年　03200968 松山夏樹

In [76]:
import numpy as np
import math
from autograd import grad # 勾配を求めるライブラリ

## (1)の関数

In [77]:
A1 = np.array([[9,12,-6,-3],[12,41,2,11],[-6,2,24,-8],[-3,11,-8,62]])
b1 = np.array([-27,-42,32,-23])
c1 = 163

def f1(x):
    return np.dot(x.T, np.dot(A1,x)) / 2 + np.dot(b1.T,x) + c1

#勾配関数
grad_f1 = grad(f1)

# 真の解
print('answer')
print(-np.dot(np.linalg.inv(A1),b1))

answer
[ 2.22777778  0.31666667 -0.69166667  0.33333333]


## (2)の関数

In [78]:
A2 = np.array([[16,8,12,-12],[8,29,16,9],[12,16,29,-19],[-12,9,-19,35]])
b2 = np.array([7,5,-2,9])
c2 = 5

def f2(x):
  return np.dot(x.T, np.dot(A2,x)) / 2 + np.dot(b2.T,x) + c2

#勾配関数
grad_f2 = grad(f2)

# 真の解
print('answer')
print(-np.dot(np.linalg.inv(A2),b2))

answer
[-3.42734375  5.24375    -5.009375   -5.5       ]


## (3)の関数

In [79]:
def f3(x):
  return (x[0]-1)**2 + 10*((x[0]**2-x[1])**2)

#勾配関数
grad_f3 = grad(f3)

# ラインサーチ
- 今回は以前の演習で使った黄金分割法を使用します。

In [80]:
# そのまま実行してください
def linesearch(x,s,func):
  # x = x + alpha*s のalphaを-1~2とする。
  a = x
  b = x + 2 * s
  tau = (-1 + np.sqrt(5)) / 2
  l1 = b - (b - a) * tau
  l2 = (b - a) * tau + a
  f1 = func(l1)
  f2 = func(l2)
  for i in range(10000):
    if np.linalg.norm(l2 - l1, ord=2) < 0.001:
      break
    if f1 < f2:
      b = l2
      l2 = l1
      l1 = a + (b - l2)
      f2 = f1
      f1 = func(l1)
    else:
      a = l1
      l1 = l2
      l2 = b - (l1 - a)
      f1 = f2
      f2 = func(l2)
  return (l1 + l2) / 2

# 演習
- 配布された問題資料を見ながら、以下の3つのアルゴリズム最急降下法、共役勾配法、準ニュートン法を完成させてください。
- また、それぞれのアルゴリズムについて配布資料の問題(1)(2)(3)の実行をしてください。
- 参考: [numpyについて](https://note.nkmk.me/python-numpy-matrix/)

## 最急降下法、共役勾配法の解の更新について
- x ← x + alpha * s と更新する alpha の決め方について以下の2通りでの方針があります。
  1. Armijoの条件を満たす最大の値をalphaとする
  2. f(x+αs) を最小とするαについてラインサーチする (厳密直線探索)
- これについてはご自身にとって分かりやすい解法を選択してください。1の解法を利用する場合、準備で定義したラインサーチは必要ありません。
- 講義資料: [アルミホの条件について](https://drive.google.com/file/d/12ZQA5ZQDN5oBMOTlxyInMqh4h8egQdCB/view?usp=sharing)

# 課題1. 最急降下法
## 実装課題
- 解の更新を実装してください。

アルミホ条件を使わず、黄金分割法のlinesearchを使う場合は、 b = x + 2 * s のところをb = x + 0.01 * sに変えると収束が早くなります 

In [6]:
class GradientDescent:
    def __init__(self, fun, der, xi=0.5, tau=0.9, tol=1e-6, itr_max=2000):
        self.fun = fun         # 目的関数
        self.der = der         # 関数の勾配
        self.xi  = xi          # Armijo条件の定数
        self.tau = tau         # Armijo条件のαの学習率
        self.tol = tol         # 収束判定条件
        self.path = None       # 解の点列
        self.itr_max = itr_max # 最大反復回数
        
    def minimize(self, x):
        path = [x]
        for i in range(self.itr_max):
            s = - self.der(x)
            print("s:",s)
            # Armijo条件を満たすような最大のalphaを求める。または f(x+αs)を最小にするalphaをラインサーチで求める。
            # WRITE ME
            # Armijo条件を満たすような最大のalphaを求める
            alpha = 1
            while self.fun(x+alpha*s)>(self.fun(x)-self.xi*alpha*np.dot(s,s)):
                alpha = self.tau*alpha
                #x = x - alpha * grad
                #path.append(x)
            
            # alphaをステップ幅とし、xを更新する
            next_x = x + alpha*s
            if np.linalg.norm(s, ord=2) < self.tol:
                x = next_x
                break
            else:
                x = next_x
                path.append(x)


        self.opt_x = x                # 最適解
        self.opt_result = self.fun(x) # 関数の最小値
        self.path = np.array(path)    # 探索解の推移

In [7]:
# (1)
gd = GradientDescent(f1, grad_f1) # 問題のインスタンス化
init = np.array([0.0,0.0,0.0,0.0]) # 初期解の設定
gd.minimize(init) # 求解

print(gd.opt_x) # 最適解
print(gd.path.size // gd.path.ndim) # 反復回数

s: [ 27.  42. -32.  23.]
s: [  9.36026316  -3.31587568 -11.11617362 -18.82847944]
s: [ 5.9613831   1.81330088 -7.48778799  4.34631807]
s: [ 3.27550145 -2.66023491 -1.31452708 -3.95004503]
s: [ 2.88589549 -0.05047384 -0.75307953  2.21092398]
s: [ 2.08745829 -1.95188735  1.07287185 -2.38640179]
s: [ 2.1756243  -0.17028389  0.43280365  1.76497446]
s: [ 1.76674179 -1.84075744  1.15882946 -2.37096094]
s: [ 1.88897315 -0.25859338  0.49992185  1.31551639]
s: [ 1.48474191 -1.85979716  1.10334355 -2.69979242]
s: [ 1.62115619 -0.29182266  0.45644199  0.97035324]
s: [ 1.1873207  -1.81344052  1.02428339 -2.89328177]
s: [ 1.32759546 -0.33731077  0.41698207  0.57345188]
s: [ 0.77549007 -1.72468637  0.90032277 -3.10761236]
s: [ 0.9202825  -0.33839695  0.33323451  0.16151349]
s: [-0.00327242 -0.13232126  0.06377725 -0.29804812]
s: [ 0.01307266 -0.00497781  0.00953095  0.00399801]
s: [ 0.01593468 -0.00899947 -0.01801452 -0.0157409 ]
s: [ 0.01062999  0.00176246 -0.00633234  0.01147724]
s: [ 0.00791798 -

In [8]:
# (2)
gd = GradientDescent(f2, grad_f2) # 問題のインスタンス化
init = np.array([0.0,0.0,0.0,0.0]) # 初期解の設定
gd.minimize(init) # 求解

print(gd.opt_x) # 最適解
print(gd.path.size // gd.path.ndim) # 反復回数

s: [-7. -5.  2. -9.]
s: [-6.31326324  3.58420955 -0.23189448  1.7817672 ]
s: [-1.76787244  0.48675525  2.54662564 -6.45082615]
s: [-3.85994734  0.92076493 -2.03393504 -0.22874413]
s: [-0.27764347  2.74730826  2.02737847 -4.2438921 ]
s: [-2.36808559  1.13216128 -2.02893052 -0.66190094]
s: [ 0.87942892  2.89330895  2.03188199 -4.5378053 ]
s: [-1.47362239  1.21831952 -2.06355738 -0.84882316]
s: [ 1.63477553  3.09092042  2.52174169 -5.02284292]
s: [-0.94583501  1.30522229 -1.81465108 -1.09018615]
s: [ 3.25457031  3.99185716  5.20774211 -7.18403517]
s: [-0.56726198  1.35645895 -1.20462777 -1.37816248]
s: [-4.4251584  -1.76524802 -7.35170901  4.95432249]
s: [-0.60517229  0.85641915 -0.96837848 -0.79869978]
s: [ 4.26237549  3.5439958   5.49866253 -6.21323825]
s: [ 0.36855251  0.76955282 -0.83026813 -0.6097978 ]
s: [-2.02273795 -0.88087598 -1.9089998   0.16915271]
s: [-0.71803316  0.55359815  0.05654877 -1.01770378]
s: [-1.18317405  0.38963604 -1.63589244  0.82452437]
s: [-0.26390657  0.732688

In [9]:
# (3)
gd = GradientDescent(f3, grad_f3) # 問題のインスタンス化
init = np.array([0.0,0.0]) # 初期解の設定
gd.minimize(init) # 求解

print(gd.opt_x) # 最適解
print(gd.path.size // gd.path.ndim) # 反復回数

s: [ 2. -0.]
s: [-0.77726742  2.74694706]
s: [0.98261455 0.47912283]
s: [-1.02063402  1.87134917]
s: [0.5887169  0.41002103]
s: [-1.0360262   1.44187376]
s: [0.38585321 0.34365972]
s: [-1.20505934  1.25826262]
s: [0.21879144 0.29103298]
s: [-1.32551542  0.95771146]
s: [0.00922386 0.19427308]
s: [ 0.35497366 -0.01377801]
s: [0.00806427 0.18591053]
s: [ 0.34383862 -0.01454493]
s: [0.03685327 0.16109617]
s: [ 0.47835629 -0.10529797]
s: [0.04670091 0.13997771]
s: [ 0.59981762 -0.19522033]
s: [0.06609937 0.10244284]
s: [-0.65311548  0.41637376]
s: [0.02194668 0.06398452]
s: [ 0.33815478 -0.11420453]
s: [0.02734957 0.04739221]
s: [-0.21854753  0.14470735]
s: [-0.00072147  0.03457805]
s: [0.05832043 0.0040791 ]
s: [0.00076827 0.03301153]
s: [6.45477731e-02 1.48379163e-05]
s: [0.00504365 0.0299371 ]
s: [ 0.08943117 -0.01419143]
s: [0.00909865 0.02623774]
s: [ 0.14445152 -0.04737828]
s: [0.01732997 0.01642277]
s: [-0.0397216   0.04419049]
s: [0.01240923 0.01794935]
s: [-0.09376752  0.06707329]


# 課題2. 共役勾配法
## 実装課題
- 課題1の場合と同じように解の更新を実装してください。興味のある方は手法を課題1と変えてくれても構いません。
- 配布資料の更新式を参考に、探索方向sの更新を行ってください。

In [29]:
#何か違う
class ConjectureDescent:
    def __init__(self, fun, der, xi=0.5, tau=0.9, tol=1e-6, itr_max=2000):
        self.fun = fun         # 目的関数
        self.der = der         # 関数の勾配
        self.xi  = xi          # Armijo条件の定数
        self.tau = tau          # Armijo条件のαの学習率
        self.tol = tol         # 収束判定条件
        self.path = None       # 解の点列
        self.itr_max = itr_max # 最大反復回数

    def minimize(self, x):
        path = [x]
        s = - self.der(x)
        for i in range(self.itr_max):
            # Armijo条件を満たすような最大のalphaを求める。 または f(x+αs)を最小にするalphaをラインサーチで求める。
            # WRITE ME
            alpha = 1
            while self.fun(x+alpha*s)>(self.fun(x)-self.xi*alpha*np.dot(s,s)):
                alpha = self.tau*alpha
                #x = x - alpha * grad
                #path.append(x)
            # alphaをステップ幅とし、xを更新する
            next_x = x + alpha*s
            if np.linalg.norm(next_x - x, ord=2) < self.tol:
                x = next_x
                break
            else:
                # 探索方向sを更新する
                # WRITE ME
                s = - self.der(next_x) + np.dot(s,(np.dot(self.der(next_x).T,self.der(next_x))/np.dot(self.der(x).T,self.der(x))))
                print("s:",s)
                #print(type(s))
                x = next_x
                path.append(x)

        self.opt_x = x                # 最適解
        self.opt_result = self.fun(x) # 関数の最小値
        self.path = np.array(path)    # 探索解の推移

In [30]:
# (1)
cd = ConjectureDescent(f1, grad_f1) # 問題のインスタンス化
init = np.array([2.0,0.3,-1.0,0.0]) # 初期解の設定
cd.minimize(init) # 求解
print(cd.opt_x )# 最適解
print(cd.path.size // cd.path.ndim) # 反復回数

s: [-0.82619027  0.5037368   4.15702349  1.38907889]
s: [-0.03282865 -0.79203151  1.96669492 -0.98298652]
s: [ 0.36113917 -0.54817384  0.74917554 -0.41234842]
s: [ 0.63576588 -0.49645482  0.65692606 -0.06739258]
[ 1.96909791  0.41399053 -0.78966101  0.28833656]
10


In [75]:
# (2)
cd = ConjectureDescent(f2, grad_f2) # 問題のインスタンス化
init = np.array([-9.0,0.0,4.5,1.0]) # 初期解の設定
cd.minimize(init) # 求解

print(cd.opt_x)# 最適解
print(cd.path.size // cd.path.ndim)# 反復回数

s: [ 75.81031509 -12.67232952 -61.56608935   3.48740615]
s: [ 50.01109224  11.19216104 -46.72320497  25.56979885]
s: [ 50.88798975  19.62806961 -40.63577403  25.8237792 ]
[-3.53000799 -0.67245606  2.10258501 -0.49381275]
8


In [38]:
# (3)
cd = ConjectureDescent(f3, grad_f3) # 問題のインスタンス化
init = np.array([0.0,0.0]) # 初期解の設定
cd.minimize(init) # 求解

print(cd.opt_x) # 最適解
print(cd.path.size // cd.path.ndim) # 反復回数

s: [3.29766397 2.74694706]
[3.70604038e-01 1.14200674e-16]
2


# 課題3. 準ニュートン法
## 実装課題
- 資料にあるBFGS更新式を参考にヘッセ行列の更新式を実装してください。

## 注意点
- 以下のような不都合が生じる場合があるので注意してください。
  - np.array([0,0,0,0]) は 1×4の1次元の行列とは認識されないことに注意してください。
  - つまり、x = np.array([0,0,0,0]) として、np.dot(x.T, x)をすると 4×1と1×4で4×4の行列が返されるのが期待する結果ですが、上記の理由により内積が返されます。
  - これを防ぐためには x.rehape(1,-1) または x.reshape(-1,1) を実行して、明示的に一次元の行列に変換する必要があります。 

In [81]:
class QuasiNewton:
    def __init__(self, fun, der, tol=1e-6, itr_max=2000):
        self.fun = fun#目的関数
        self.der = der#関数の勾配
        self.tol = tol#収束判定条件
        self.path = None#解の点列
        self.itr_max = itr_max#最大反復回数
        
    def minimize(self,x,H):
        path=[x]
        for i in range(self.itr_max):
            s = - np.dot(np.linalg.inv(H), self.der(x))
            next_x = x + s
            if np.linalg.norm(next_x - x, ord=2) < self.tol:
                break
            else:
                #BFGS更新式を用いてHを更新
                #WRITE ME
                y=self.der(next_x)-self.der(x)
                H=H-(np.dot(H,np.dot(s.reshape(-1,1),np.dot(s.reshape(1,-1),H))))/np.dot(s.reshape(1,-1),np.dot(H,s.reshape(-1,1)))\
                +np.dot(y.reshape(-1,1),y.reshape(1,-1))/np.dot(s.reshape(1,-1),y.reshape(-1,1))
               
                x = next_x
                path.append(x)
        self.opt_x=x
        #最適解
        self.opt_result=self.fun(x)
        #関数の最小値
        self.path=np.array(path)
        #探索解の推移

In [82]:
# (1)
qn = QuasiNewton(f1, grad_f1) # 問題のインスタンス化
init_x = np.array([1.0,1.0,5.0,3.0])
init_H = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
qn.minimize(init_x, init_H)

print(qn.opt_x)
print(qn.path.size // qn.path.ndim)

[ 2.22777777  0.31666669 -0.69166666  0.33333339]
20


In [12]:
# (2)
qn = QuasiNewton(f2, grad_f2) # 問題のインスタンス化
init_x = np.array([0.0,0.0,0.0,0.0])
init_H = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
qn.minimize(init_x, init_H)

print(qn.opt_x)
print(qn.path.size // qn.path.ndim)

[-3.42734382  5.24375022 -5.00937515 -5.50000019]
28


In [13]:
# (3)
qn = QuasiNewton(f3, grad_f3) # 問題のインスタンス化
init_x = np.array([0.0,0.0])
init_H = np.array([[1,0],[0,1]])
qn.minimize(init_x, init_H)

print(qn.opt_x)
print(qn.path.size // qn.path.ndim)

[1.         0.99999999]
22


考察

最急降下法は、反復回数が10^2のオーダーであるなど収束が非常に遅い。一方で正確な計算結果が得られている。

共役勾配法は、今回どれも局所解に陥っており初期値に対してナイーヴであることがわかった。一方で、探索回数が10^0~10^1のオーダーであり非常に小さいことが同時に確認された。

準ニュートン法は、うまく初期値を与えなければ収束しない。
ただし、良い初期値を選べば高速に収束する(超1次収束)ことが確認された。
